### 🌌**LangGraph Conversation Example**

In this notebook, we will build an interactive conversation flow using LangGraph and OpenAI's GPT-4.

**Objectives:**
- Learn how to define a state with LangGraph
- Add a Language Model (LLM) as a node in the graph
- Implement a Human-in-the-Loop step where the user provides feedback
- Define edges to connect nodes and control the flow of conversation
- Compile and run the graph to simulate the conversation

**Let's get started!**


In [ ]:
%pip install langgraph langchain-openai typing-extensions

### **1. Defining the State**

In LangGraph, the **State** is the container for all data passed between nodes.  
Here, we define the state structure with:
- `messages`: A list of messages exchanged in the conversation.
- `end`: A boolean flag to indicate when to stop the graph.

The `State` class is defined using `TypedDict` for type safety and clarity.

In [ ]:

from typing_extensions import TypedDict
from typing import Annotated
from langgraph.graph.message import add_messages

class State(TypedDict):
    messages: Annotated[list, add_messages]
    end: bool

### **2. Creating the Graph**

A **StateGraph** is the core structure where nodes and edges are defined.  
We initialize the graph with our previously defined `State`, which will handle the flow of messages and control the termination of the conversation.

In [ ]:
from langgraph.graph import StateGraph

graph_builder = StateGraph(State)

### **3. Setting Up the LLM**

The **Language Model (LLM)** is responsible for generating responses in the conversation.  
Here, we set up `ChatOpenAI` with the `gpt-4o-mini` model. The function `llm_step` uses the LLM to generate a response based on the current state of the conversation.

This step is added as a node in the graph, making it part of the flow.


In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

def llm_step(state: State):
    response = llm.invoke(state["messages"])
    return {"messages": [response]}

graph_builder.add_node("llm", llm_step)

### **4. Adding Human-in-the-Loop Step**

A **Human-in-the-Loop** step allows user feedback to guide the conversation.  
Here, we present the LLM's response to the user, who can:
- Type feedback, which becomes the next message in the conversation.
- Type "stop" to end the conversation, setting the `end` flag to `True`.

This node is added as part of the graph to involve human feedback.


In [ ]:
def human_review(state: State):
    last_message = state["messages"][-1].content
    print(f"\n LLM says: {last_message}")
    human_input = input("Your feedback (type 'stop' to end, or ENTER to continue): ")

    if human_input.strip().lower() == "stop":
        return {"end": True}  # Set flag to end
    elif human_input.strip():
        return {"messages": [("user", human_input)]}
    else:
        return {"messages": []}

graph_builder.add_node("human", human_review)

### **5. Setting Up Edges**

In LangGraph, **edges** define how nodes are connected.  
We create two types of edges:
- A regular edge from the `llm` node to the `human` node.
- Conditional edges based on the `end` flag. If `end` is `True`, the conversation ends; otherwise, it loops back to the `llm` node.

We also define an empty `END` node that marks the end of the graph.


In [ ]:
graph_builder.add_edge("llm", "human")
graph_builder.add_conditional_edges(
    "human",
    lambda state: "END" if state.get("end") else "llm",
)
graph_builder.add_node("END", lambda state: state)  # Empty end node

graph_builder.set_entry_point("llm")

### **6. Compiling the Graph**

After defining nodes and edges, we **compile** the graph into a runnable structure.  
This step ensures that LangGraph knows the exact sequence of operations to perform when the graph is executed.


In [ ]:
graph = graph_builder.compile()

### **7. Running the Graph**

Finally, we **run the graph** by providing an initial user input.  
LangGraph will execute the steps, call nodes, update the state, and stream the conversation as it unfolds.  
Once the conversation is complete, we display all the messages, showing a full interaction between the user and the AI.


In [ ]:
user_input = input("Your initial input: ")

events = graph.stream(
    {"messages": [("user", user_input)], "end": False},
    config={"configurable": {"thread_id": "1"}},
    stream_mode="values"
)

# Collect conversation steps and output once at the end
conversation = []

for event in events:
    if "messages" in event:
        for message in event["messages"]:
            # Append conversation with proper format
            role = "Human" if message.type == "user" else "AI"
            conversation.append(f"\n================================ {role} Message =================================")
            conversation.append(message.content)

# Display the full conversation at once
print("".join(conversation))

print("\n✅ Conversation ended!")
